In [3]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import pandas as pd
modelname = "medicalai/ClinicalBERT"
bertTokenizer = AutoTokenizer.from_pretrained(modelname)
bertModel = AutoModel.from_pretrained(modelname)
#https://huggingface.co/medicalai/ClinicalBERT

Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
batch_size = 25
start = 600

In [5]:
start = 600
dataset = pd.read_csv('file.csv')
dataset = dataset.dropna()
raw_inputs = [row[0] for row in dataset.values.tolist()]
batch_of_inputs = raw_inputs[start:start+batch_size]
inputs = bertTokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt", max_length=512)
outputs = bertModel(**inputs)
embeddings = torch.empty(batch_size, 768)
for i in range(batch_size):
  average = torch.mean(outputs.last_hidden_state[i], dim=0)
  embeddings[i]=average
t_np = embeddings.numpy() #convert to Numpy array
df = pd.DataFrame(t_np) #convert to a dataframe
df.to_csv("embeddings6.0",index=False) #save to file


In [11]:
def calculate_similarity (tensor_a, tensor_b):
  # Calculate the dot product of the two tensors
  dot_product = torch.dot(tensor_a, tensor_b)

  # Calculate the magnitudes of the two tensors
  magnitude_a = torch.norm(tensor_a)
  magnitude_b = torch.norm(tensor_b)

  # Calculate the cosine similarity
  cosine_similarity = dot_product / (magnitude_a * magnitude_b)

  # Print the cosine similarity
  return(cosine_similarity)


In [13]:
closestidx = 0
closeness = 0
questionidx = 78
questionTokens = bertTokenizer(dataset['question'].tolist()[questionidx], padding=True, truncation=True, return_tensors="pt", max_length=512)
questionOutput = bertModel(**questionTokens)
questionEmbedding = torch.mean(questionOutput.last_hidden_state[0], dim=0)
for i in range(20):
  if calculate_similarity(embeddings[i], questionEmbedding) > closeness:
    closeness = calculate_similarity(embeddings[i], questionEmbedding)
    closestidx = i
print(closestidx)
#print(dataset['answer'].tolist()[closestidx])


4
